In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import qmc
from scipy.spatial import ConvexHull
from farthest_voronoi_diagram import FarthestVoronoiDiagram
from smallest_separating_circle import (
    find_minimal_separating_circle,
    find_maximal_surrounding_circle,
)

In [ ]:
# Toggle show/hide
show_convex_hull = True
show_farthese_voronoi = True
show_surrounding_circle = True
show_separating_circle = True
show_event_points = True

# Data generation method, could be 'demo', 'sobol', 'gaussian' or 'halton', or write your own
generation_method = "demo"
red_points, blue_points = None, None

# Number of points to generate, both red and blue
number_of_points = 5, 5

# The shift of the center of blue points compared to red points
point_shift = np.array([0.05, -1.015])

In [ ]:
# Data generation

if generation_method == "gaussian":
    rng = np.random.default_rng()
    red_points = rng.normal(np.zeros(2), 0.5, size=(number_of_points[0], 2))
    blue_points = rng.normal(point_shift, 0.5, size=(number_of_points[1], 2))
elif generation_method == "sobol":
    sampler = qmc.Sobol(d=2, seed=np.random.default_rng())
    red_points = sampler.random(number_of_points[0])
    blue_points = sampler.random(number_of_points[1]) + point_shift
elif generation_method == "halton":
    sampler = qmc.Halton(d=2, seed=np.random.default_rng())
    red_points = sampler.random(number_of_points[0])
    blue_points = sampler.random(number_of_points[1]) + point_shift
elif generation_method == "demo":
    red_points = np.array([(2, 4), (4, 0), (4, 5.25), (4, 4), (4, 2), (6, 2)])
    blue_points = np.array(
        [(2.2, 2.5), (-2, 2), (0, 0), (1, 1), (-0.7, -1), (-1.6, -2), (-1, -4)]
    )

# else write your own data in the cell above

In [ ]:
plt.axis("equal")
# plt.gca().set_xlim(-4, 12)
# plt.gca().set_ylim(-4.5, 10)

# Red/blue points
plt.plot(red_points[:, 0], red_points[:, 1], "ro")
plt.plot(blue_points[:, 0], blue_points[:, 1], "bo")

if show_convex_hull:
    red_hull = ConvexHull(red_points)
    for simplex in red_hull.simplices:
        plt.plot(red_points[simplex, 0], red_points[simplex, 1], "r-")

if show_surrounding_circle:
    surrounding_circle = find_maximal_surrounding_circle(red_points)
    surrounding_circle_plt = plt.Circle(
        surrounding_circle.center, surrounding_circle.radius, color="r", fill=False
    )
    plt.gca().add_artist(surrounding_circle_plt)

if show_farthese_voronoi:
    fvd = FarthestVoronoiDiagram.from_points(red_points)
    for edge in fvd.edges:
        ei, ej = edge.origin_vert.point, edge.dest_vert.point

        xs = [ei[0], ej[0]]
        ys = [ei[1], ej[1]]
        plt.plot(xs, ys, "g-")

if show_separating_circle:
    separating_circle, event_points = find_minimal_separating_circle(red_points, blue_points)
    separating_circle_plt = plt.Circle(
        separating_circle.center, separating_circle.radius, color="y", fill=False
    )
    plt.gca().add_artist(separating_circle_plt)
    if show_event_points:
        for p in event_points:
            plt.plot(p[0], p[1], "go")

x_min = min([p[0] for p in red_points] + [p[0] for p in blue_points]) - 1.
y_min = min([p[1] for p in red_points] + [p[1] for p in blue_points]) - 1.
x_max = max([p[0] for p in red_points] + [p[0] for p in blue_points]) + 1.
y_max = max([p[1] for p in red_points] + [p[1] for p in blue_points]) + 1.
plt.gca().set_xlim(x_min, x_max)
plt.gca().set_ylim(y_min, y_max)

plt.show()